# Machine Learning for Reaserch

[Institute of Data Science at Maastricht University](https://www.maastrichtuniversity.nl/research/institute-data-science)


This notebook extends a notebook used during a machine learning workshop by the institute of data science at Maastricht University.
In order to advance my understanding of machine learning, I will try several techniques as I attempt to answer the following questions:

1. Which clusters of chronic conditions can be identified using these data?
2. Can we predict the presence of chronic illness?

The first question will be approached using *unsupervised* learning methods. The second question will be approached using *supervised* learning methods.

Several people will work on these questions independently and results will be discussed afterwards during a joint session.

## Steps to choose a method based in a research question

- 1) Get to know your data  
- 2) Fit on train data  
- 3) Evaluate on test data  
- 4) Compare Methods  

In [109]:
import os
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

%matplotlib inline

pd.set_option("max_columns", 999)

### Load Across National Health Data
- Terms of use [ICPSR 34241](https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/34241/terms)

This Across-national Health Outcome dataset includes over 30000 participants' personal social, economic, life-style, health data across 6 countries -- India, Ghana, China, South Africa, Mexico, and Russia. Features contains income, education level, parents socio-economic level, drinking, smoking, self-report health status, chronic disease, etc.
Dataset contains categorial value: 0 or 1 or -1 (-1 means value is missing), numercial values. 


This data is a part of Research on Early Life and Aging Trends and Effects:
McEniry, Mary. Research on Early Life and Aging Trends and Effects (RELATE): A Cross-National Study. Ann Arbor, MI: Inter-university Consortium for Political and Social Research [distributor], 2015-05-07. https://doi.org/10.3886/ICPSR34241.v2 

In [137]:
output = "../data/across-national-health-data.csv"

if not os.path.exists(output):
    df = (pd.read_csv('https://s3.eu-west-3.amazonaws.com/pedrohserrano-datasets/across-national-health-data.csv')
         .rename(columns=str.lower))
    df.to_csv(output, index=False)
else:
    df = (pd.read_csv(output)
          .rename(columns=str.lower))

df = df.set_index("id")

In [138]:
df.head()

,country,birthcohort,residence,gender,age,civil,positionfin,yeduca,education,income,ppp,quint,wealth_index,wealth_assets,hhsize,hscore,regime,ruralfin,birthmonth,birthyear,mothereduc_gen,motheroccup_sage,height,height25new,stunt,fathereduc_gen,fatheroccup_sage,smoke,smoke_gen,exercise,exercise_rigor,drink,healthuse,health_sage,poorhealth,goodhealth,nadlgt0,nadlgt0_alt,niadlgt0,nadlgt0harm,bath,dress,toilet,transfer,bow,hypertension,respiratory,heart,angina,heart_attack,diabetes,stroke,arthritis,nchronic,weight,waist,hip,bmi,bmi_cat,obesity
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3323,Ghana-WHO/SAGE,15,0,1,50,2,1,0,2,,,-1,2,3,6,135,5,0,3,1957,1,3,172.4,0,0,1,3,0,0,1,1,1,0,2,0,83.3333,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,76,104,106,25.57049,4,0
11580,China-WHO/SAGE,12,0,1,66,1,1,2,2,20000,8294.836,5,3,7,2,144,5,1,1,1941,1,5,163,0,0,1,5,0,0,0,0,0,1,4,1,66.6667,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,1,65.5,88,100,24.65279,3,0
5593,China-WHO/SAGE,12,1,1,65,1,3,0,1,2333.333,967.7308,2,1,1,3,144,5,1,7,1942,1,3,136.2,1,1,1,3,0,0,0,0,0,1,4,1,50,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,32,64.2,84.4,17.25027,2,0
8744,China-WHO/SAGE,12,0,0,68,1,2,6,2,30000,12442.25,5,3,7,2,144,5,0,10,1942,1,5,166,0,0,2,5,0,0,0,0,1,0,2,0,83.3333,0,1,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,75,95,102,27.2173,4,0
6277,China-WHO/SAGE,14,1,0,55,1,1,6,2,5000,2073.709,3,3,6,2,144,5,1,2,1952,1,3,170.8,0,0,1,3,1,2,0,0,0,1,3,0,100,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,69,76.5,90.5,23.6523,3,0


---
## Get to know your data

The user guide can be found as pdf in "../34241-User_guide.pdf"

In [139]:
# a look at the numeric variables
df.describe()

,birthcohort,residence,gender,civil,positionfin,education,quint,wealth_index,wealth_assets,hscore,regime,ruralfin,birthmonth,birthyear,mothereduc_gen,motheroccup_sage,height25new,stunt,fathereduc_gen,fatheroccup_sage,smoke,smoke_gen,exercise,exercise_rigor,drink,healthuse,health_sage,poorhealth,nadlgt0,nadlgt0_alt,niadlgt0,nadlgt0harm,bath,dress,toilet,transfer,bow,hypertension,respiratory,heart,angina,heart_attack,diabetes,stroke,arthritis,nchronic,bmi_cat,obesity
count,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.0,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000
mean,11.315176,0.445169,0.434964,1.619404,1.323605,1.796978,2.307462,2.204665,3.995679,129.168297,4.567581,0.352419,3.973360,1809.558224,0.924983,3.010073,0.066799,-0.098661,1.064944,2.337124,0.201087,0.443101,0.116024,0.116024,0.207237,0.239496,2.438887,0.066137,0.029874,0.257734,0.088535,0.013810,-0.062425,-0.069344,-0.055136,-0.018635,0.099934,0.128774,-0.055162,-0.133121,-0.133121,-1.0,-0.047634,-0.084904,0.071146,0.554672,2.719841,-0.029264
std,3.868191,0.558426,0.620141,1.500357,0.891373,1.385564,1.975064,1.077217,2.836785,29.353722,0.669696,0.706874,4.515658,493.609185,1.040324,2.042169,0.604644,0.441606,1.226787,1.820011,0.629282,0.982930,0.579927,0.579927,0.633633,0.828899,1.438599,0.534713,0.521140,0.725957,0.635814,0.498003,0.403305,0.391180,0.413474,0.456341,0.568247,0.592347,0.418219,0.604201,0.604201,0.0,0.428834,0.374792,0.545974,1.094957,1.813441,0.525959
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,61.000000,3.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-8.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,10.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,112.000000,4.000000,0.000000,-1.000000,1935.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.0,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,12.000000,0.000000,1.000000,1.000000,1.000000,2.000000,2.000000,3.000000,4.000000,135.000000,5.000000,0.000000,3.000000,1945.000000,1.000000,3.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
75%,14.000000,1.000000,1.000000,2.000000,2.000000,3.000000,4.000000,3.000000,7.000000,144.000000,5.000000,1.000000,8.000000,1952.000000,1.000000,5.000000,0.000000,0.000000,2.000000,3.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,3.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0,0.000000,0.000000,0.000000,1.000000,4.000000,0.000000
max,16.000000,1.000000,1.000000,5.000000,3.000000,4.000000,5.000000,3.000000,8.000000,175.000000,5.000000,1.000000,12.000000,1978.000000,4.000000,5.000000,1.000000,1.000000,4.000000,5.000000,1.000000,2.000000,1.000000,1.00000

In [140]:
# what about categorical variables?
df.select_dtypes(include=[object]).head()

,country,age,yeduca,income,ppp,hhsize,height,goodhealth,weight,waist,hip,bmi
id,,,,,,,,,,,,
3323,Ghana-WHO/SAGE,50,0,,,6,172.4,83.3333,76,104,106,25.57049
11580,China-WHO/SAGE,66,2,20000,8294.836,2,163,66.6667,65.5,88,100,24.65279
5593,China-WHO/SAGE,65,0,2333.333,967.7308,3,136.2,50,32,64.2,84.4,17.25027
8744,China-WHO/SAGE,68,6,30000,12442.25,2,166,83.3333,75,95,102,27.2173
6277,China-WHO/SAGE,55,6,5000,2073.709,2,170.8,100,69,76.5,90.5,23.6523


In [141]:
# looks like those should alse be numerical, except for country
categorical_vars = df.select_dtypes(include=[object]).columns.tolist()
categorical_vars.remove("country")

for catvar in categorical_vars:
    df[catvar] = pd.to_numeric(df[catvar], errors="coerce")

df.loc[:, categorical_vars].dtypes
df.loc[:, categorical_vars].head()


,age,yeduca,income,ppp,hhsize,height,goodhealth,weight,waist,hip,bmi
id,,,,,,,,,,,
3323,50.0,0.0,NaN,NaN,6.0,172.4,83.3333,76.0,104.0,106.0,25.57049
11580,66.0,2.0,20000.000,8294.8360,2.0,163.0,66.6667,65.5,88.0,100.0,24.65279
5593,65.0,0.0,2333.333,967.7308,3.0,136.2,50.0000,32.0,64.2,84.4,17.25027
8744,68.0,6.0,30000.000,12442.2500,2.0,166.0,83.3333,75.0,95.0,102.0,27.21730
6277,55.0,6.0,5000.000,2073.7090,2.0,170.8,100.0000,69.0,76.5,90.5,23.65230


In [142]:
# now let's look at ALL numeric variables
df.describe()

,birthcohort,residence,gender,age,civil,positionfin,yeduca,education,income,ppp,quint,wealth_index,wealth_assets,hhsize,hscore,regime,ruralfin,birthmonth,birthyear,mothereduc_gen,motheroccup_sage,height,height25new,stunt,fathereduc_gen,fatheroccup_sage,smoke,smoke_gen,exercise,exercise_rigor,drink,healthuse,health_sage,poorhealth,goodhealth,nadlgt0,nadlgt0_alt,niadlgt0,nadlgt0harm,bath,dress,toilet,transfer,bow,hypertension,respiratory,heart,angina,heart_attack,diabetes,stroke,arthritis,nchronic,weight,waist,hip,bmi,bmi_cat,obesity
count,37725.000000,37725.000000,37725.000000,35116.000000,37725.000000,37725.000000,33197.000000,37725.000000,3.131700e+04,31317.000000,37725.000000,37725.000000,37725.000000,36226.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,32005.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,33494.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.000000,37725.0,37725.000000,37725.000000,37725.000000,37725.000000,32070.000000,31540.000000,31484.000000,31939.000000,37725.000000,37725.000000
mean,11.315176,0.445169,0.434964,63.550148,1.619404,1.323605,5.521342,1.796978,1.498901e+05,3181.859343,2.307462,2.204665,3.995679,4.125573,129.168297,4.567581,0.352419,3.973360,1809.558224,0.924983,3.010073,159.116322,0.066799,-0.098661,1.064944,2.337124,0.201087,0.443101,0.116024,0.116024,0.207237,0.239496,2.438887,0.066137,78.592183,0.029874,0.257734,0.088535,0.013810,-0.062425,-0.069344,-0.055136,-0.018635,0.099934,0.128774,-0.055162,-0.133121,-0.133121,-1.0,-0.047634,-0.084904,0.071146,0.554672,62.671690,86.380489,95.373565,24.810260,2.719841,-0.029264
std,3.868191,0.558426,0.620141,9.800265,1.500357,0.891373,5.074384,1.385564,6.298205e+06,9588.530207,1.975064,1.077217,2.836785,2.996394,29.353722,0.669696,0.706874,4.515658,493.609185,1.040324,2.042169,9.970561,0.604644,0.441606,1.226787,1.820011,0.629282,0.982930,0.579927,0.579927,0.633633,0.828899,1.438599,0.534713,22.761310,0.521140,0.725957,0.635814,0.498003,0.403305,0.391180,0.413474,0.456341,0.568247,0.592347,0.418219,0.604201,0.604201,0.0,0.428834,0.374792,0.545974,1.094957,16.591431,14.361140,14.091154,7.344935,1.813441,0.525959
min,-1.000000,-1.000000,-1.000000,29.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000e+00,0.000000,-1.000000,-1.000000,-1.000000,1.000000,61.000000,3.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,80.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-8.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,24.000000,25.000000,-8.000000,8.018978,-1.000000,-1.000000
25%,10.000000,0.000000,0.000000,55.000000,1.000000,1.000000,0.000000,1.000000,2.500000e+03,567.312900,1.000000,1.000000,1.000000,2.000000,112.000000,4.000000,0.000000,-1.000000,1935.000000,1.000000,1.000000,153.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,2.000000,0.000000,66.666700,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.0,0.000000,0.000000,0.000000,0.000000,51.000000,77.375000,87.300000,20.683710,3.000000,0.000000
50%,12.000000,0.000000,1.000000,62.000000,1.000000,1.000000,5.000000,2.000000,6.666667e+03,1410.122000,2.000000,3.000000,4.000000,3.000000,135.000000,5.000000,0.000000,3.000000,1945.000000,1.000000,3.000000,159.200000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,83.333300,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

### How many features are there ?

In [143]:
# rows and features/columns
df.shape

(37725, 60)

### Is any value missing in the dataset ? 

In [144]:
def show_missing_values(df):
    
    # overview of missing values per variable
    ismiss = df.isnull().sum()
    
    # the sum of that is the total of missing values
    print("Total missing values: {}".format(ismiss.sum()))
    
    # have a closer look only if the total of missing values is not zero
    if ismiss.sum() > 0:
        missdf = pd.DataFrame(ismiss[ismiss > 0], index=(ismiss[ismiss > 0].index), columns=["missing_values"])
        missdf["missing_values_as_perc"] = round(missdf["missing_values"] / df.shape[0] * 100, 1)
        print(missdf)

        

In [145]:
# set -1 to NaN
df = df[df != -1]
show_missing_values(df)

Total missing values: 315364
                  missing_values  missing_values_as_perc
birthcohort                 2609                     6.9
residence                   1223                     3.2
gender                      2618                     6.9
age                         2609                     6.9
civil                       2819                     7.5
positionfin                 2283                     6.1
yeduca                      4528                    12.0
education                   4528                    12.0
income                      6408                    17.0
ppp                         6408                    17.0
quint                       6408                    17.0
wealth_index                1792                     4.8
wealth_assets               1792                     4.8
hhsize                      1499                     4.0
ruralfin                    5120                    13.6
birthmonth                 12606                    33.4
bi

# Data prep/cleanup/wrangling

## Entire dataset

In [146]:
# heart_attach has 100% missing, let's get rid of it
del df["heart_attack"]

In [147]:
# create binary variable to indicate chronic illness
df["ischronic"] = df["nchronic"] >= 1

# how many people are chronically ill?
df.ischronic.value_counts()

False    21779
True     15946
Name: ischronic, dtype: int64

## Subset for Question 1

In [150]:
# use a subset of the data for this question, only containing the conditions
conditions = ["hypertension",
              "respiratory",
              "heart",
              "angina",
              "diabetes",
              "stroke",
              "arthritis"]

clust = df.loc[:, conditions]

In [151]:
clust.head()

,hypertension,respiratory,heart,angina,diabetes,stroke,arthritis
id,,,,,,,
3323,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11580,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5593,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8744,0.0,0.0,NaN,NaN,0.0,0.0,0.0
6277,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [152]:
# dimensions
clust.shape

(37725, 7)

In [153]:
# how about missing values?
show_missing_values(clust)

Total missing values: 41534
              missing_values  missing_values_as_perc
hypertension            4502                    11.9
respiratory             4397                    11.7
heart                   9731                    25.8
angina                  9731                    25.8
diabetes                4410                    11.7
stroke                  4387                    11.6
arthritis               4376                    11.6


In [154]:
# it's quite rigorous, but let's remove all rows with any missing values
clust = clust.dropna()
# dimensions after removing those rows
clust.shape

(27832, 7)

# Question 1: Which clusters of chronic conditions can be identified using these data?

## Approach 1: Clustering

---
## Fit on train data

---
## Evaluate on test data

---
### Documentation of the Method

The goal of construct a method is the reproducibility, that is why is important to document the procedures, taking into account versioning control of script for replicable science.

Notebooks like this one go a long way toward making our work reproducible. Since we documented every step as we moved along, we have a written record of what we did and why we did it — both in text and code.

Beyond recording what we did, we should also document what software and hardware we used to perform our analysis. This typically goes at the top of our notebooks so our readers know what tools to use.

[Sebastian Raschka](https://sebastianraschka.com/) created a handy notebook tool for this:

In [ ]:
#!pip install watermark
#only install once

In [ ]:
%load_ext watermark

In [ ]:
%watermark -a 'Maastricht University' -nmv --packages turicreate,pandas,seaborn,matplotlib